In [1]:
# standard imports
import os
import re
import sys
import requests
import unicodedata
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# natural language processing
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# pandas display settings
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.width = None

In [3]:
# number of ngrams to consider from each blog
terms_to_consider = 50

In [4]:
# read and store all stopwords in lowercase
curr_path = os.path.abspath("cumulative_tfidf.ipynb")
stopwords_path = os.path.abspath(os.path.join(curr_path, "../../../..", "Read_Files", "stopwords_cleaned.txt"))
with open(stopwords_path) as file:
    stopwords = [line.strip().lower() for line in file]

In [5]:
# get all links
csv_path = os.path.abspath(os.path.join(curr_path, "../..", "alokeveer_analytics_data.csv"))
df = pd.read_csv(csv_path)
no_of_blogs = df.shape[0]

In [6]:
# function to get all page content from html response
def get_page_text(html_response):
    # getting page content
    html_text = html_response.text
    soup = BeautifulSoup(html_text, "lxml")

    # various sources of text
    para_text = [element.text.strip() for element in soup.find_all("p")]
    header_text = [element.text.strip() for element in soup.find_all(["h1", "h2", "h3", "h4", "h5", "h6"])]
    span_text = [element.text.strip() for element in soup.find_all("span")]
    all_text = para_text + header_text + span_text
    
    return " ".join(all_text)

In [7]:
# function to lemmatize and clean page text
def clean_page_text(text):
    # lemmatizer
    wnl = nltk.stem.WordNetLemmatizer()

    # text cleaning
    text = (unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore').lower())
    text = text.replace("/", " ")
    text = text.replace("-", " ")
    words = re.sub(r'[^\w\s]', '', text).split()

    return " ".join([wnl.lemmatize(word) for word in words if wnl.lemmatize(word) not in stopwords])

In [8]:
# function to store top scoring words from corpus
def get_top_keywords(corpus, n):
    # function to get tfidf scoring
    vectorizer = TfidfVectorizer(stop_words=stopwords, ngram_range=(n, n))
    tfIdf = vectorizer.fit_transform(corpus)
    tfIdf_T = tfIdf.T.todense()

    # get all scores
    feature_names = vectorizer.get_feature_names()
    feature_scores = [np.max(tfIdf_T[i]) for i in range(len(feature_names))]

    # store data
    df_tfidf_scores = pd.DataFrame({"keyword": feature_names, "score": feature_scores})
    df_tfidf_scores.sort_values("score", ascending=False, inplace=True)
    df_tfidf_scores.reset_index(inplace=True, drop=True)
    return df_tfidf_scores.head(terms_to_consider)["keyword"]

In [9]:
# get document corpus
successful_blog_list = []
document_corpus = []

# iterate through all blogs
for blog_link in df["Blog Link"]:
    # getting page response
    html_response = requests.get(blog_link)
    if(html_response.status_code != 200):
        continue

    # get page content
    all_text = get_page_text(html_response)

    # lemmatize and clean sentence
    clean_text = clean_page_text(all_text)

    # store data
    successful_blog_list.append(blog_link)
    document_corpus.append(clean_text)

In [10]:
# print all output
# set output path
output_path = os.path.abspath(os.path.join(curr_path, "../..", "Outputs/cumulative_tfidf.txt"))

# write to file
original_stdout = sys.stdout
with open(output_path, "w") as f:
    sys.stdout = f

    print("KEYWORDS FROM CUMULATIVE SCORING:")
    print()

    df_output = pd.DataFrame()
    df_output["1-gram"] = get_top_keywords(document_corpus, 1)
    df_output["2-gram"] = get_top_keywords(document_corpus, 2)
    df_output["3-gram"] = get_top_keywords(document_corpus, 3)
    
    print(df_output.to_string())
    print()

    # reset stdout
    sys.stdout = original_stdout

C:\Users\aloke\AppData\Roaming\Python\Python310\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
